In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from collections import Counter

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train = train.iloc[:,1:]
test = test.iloc[:,1:]

In [3]:
train.isnull().sum()[train.isnull().sum()>0], test.isnull().sum()[test.isnull().sum()>0]

(Series([], dtype: int64), Series([], dtype: int64))

In [4]:
for col in train.columns:
    print(col)
    print(train[col].unique())

AGE
[72 66 75 77 85 82 67 73 84 78 61 69 48 70 50 46 63 57 53 74 56 81 88 65
 79 62 36 34 40 59 54 68 60 44 42 83 51 49 76 55 47 71 87 80 52 31 58 37
 64 43 39 45]
Adenocarcinoma
[1 0]
Large cell carcinoma
[1 0]
Squamous cell carcinoma
[0 1]
TX
[1 0]
T0
[0 1]
T1
[0 1]
T1a
[0 1]
T1b
[0 1]
T1c
[0 1]
T2
[0 1]
T2a
[0 1]
T2b
[0 1]
T3
[0 1]
T4
[0 1]
N1
[0 1]
N2
[0 1]
N3
[0 1]
M1a
[0 1]
M1b
[0 1]
M1c
[0 1]
Type of Drink
[99  3  2  1]
Smoke
[2 1 0]
Height
[142.  179.4 169.8 158.5 172.3 153.8 146.4 144.1 151.8 171.  165.1 146.2
 142.9 167.1 166.  145.8 176.4 175.5 145.4 168.1 156.  167.2 162.  167.4
 164.8 168.3 167.6 163.6 149.  143.1 159.2 163.1 170.2 150.5 161.2 173.5
 148.5 167.7 181.6 172.1 165.7 165.3 152.2 182.9 171.9 150.8 156.2 168.5
 169.7 146.6 176.  147.7 174.1 171.5 166.2 176.5 160.2 161.9 152.8 152.
 149.7 158.  178.7 178.5 162.7 144.  166.6 165.  162.9 154.8 151.6 147.3
 154.6 166.3 156.5 150.4 152.9 148.  162.5 170.1 173.9 159.1 148.1 148.4
 168.9 157.6 180.1 163.5 174.  177.5 1

In [5]:
# Type of Drink = [99  3  2  1]
# Smoke = [2 1 0]
# EGFR mutation Detection = [99  1  0]

train = pd.get_dummies(train, columns = ['Type of Drink', 'Smoke', 'EGFR mutation Detection'])
test = pd.get_dummies(test, columns = ['Type of Drink', 'Smoke', 'EGFR mutation Detection'])


In [6]:
train

,AGE,Adenocarcinoma,Large cell carcinoma,Squamous cell carcinoma,TX,T0,T1,T1a,T1b,T1c,...,Type of Drink_1,Type of Drink_2,Type of Drink_3,Type of Drink_99,Smoke_0,Smoke_1,Smoke_2,EGFR mutation Detection_0,EGFR mutation Detection_1,EGFR mutation Detection_99
0,72,1,1,0,1,0,0,0,0,0,...,False,False,False,True,False,False,True,False,False,True
1,66,1,0,0,0,0,1,0,0,0,...,False,False,False,True,False,False,True,False,False,True
2,75,1,1,0,1,0,0,0,0,0,...,False,False,True,False,False,False,True,False,False,True
3,77,1,0,1,0,0,1,0,0,0,...,False,True,False,False,False,False,True,False,True,False
4,85,1,0,0,0,0,0,0,0,1,...,False,True,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,69,1,0,0,0,0,0,0,1,0,...,False,False,False,True,False,True,False,True,False,False
9996,37,0,0,0,0,0,0,1,0,0,...,False,True,False,False,False,True,False,True,False,False
9997,31,1,0,0,0,0,0,1,0,0,...,False,True,False,False,False,False,True,False,False,True
9998,85,0,0,0,0,0,0,0,0,0,...,True,False,False,False,False,False,True,False,True,False


In [7]:
test

,AGE,Adenocarcinoma,Large cell carcinoma,Squamous cell carcinoma,TX,T0,T1,T1a,T1b,T1c,...,Type of Drink_1,Type of Drink_2,Type of Drink_3,Type of Drink_99,Smoke_0,Smoke_1,Smoke_2,EGFR mutation Detection_0,EGFR mutation Detection_1,EGFR mutation Detection_99
0,35,1,0,0,0,0,0,0,0,0,...,True,False,False,False,False,True,False,True,False,False
1,60,1,0,0,0,0,0,0,1,0,...,False,False,True,False,True,False,False,True,False,False
2,45,1,1,1,1,0,0,0,0,0,...,True,False,False,False,True,False,False,False,False,True
3,31,0,0,0,1,0,0,0,0,0,...,False,False,False,True,False,True,False,False,True,False
4,55,0,0,1,0,0,0,1,0,0,...,False,False,False,True,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,47,1,1,0,0,0,0,0,0,1,...,False,False,False,True,True,False,False,False,False,True
4996,44,1,0,1,0,0,1,0,0,0,...,False,False,False,True,False,False,True,False,False,True
4997,80,0,0,1,0,0,0,1,0,0,...,False,False,True,False,True,False,False,True,False,False
4998,69,1,0,0,0,0,0,0,0,0,...,False,True,False,False,False,False,True,True,False,False


## 0. Testing Other models with raw data

In [ ]:
X = train.iloc[:,:-1]
y = train.iloc[:,-1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##### RandomForest

In [ ]:
rf = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Best parameters found:  {'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)

best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test, y_test)
print("Test accuracy of best model: ", accuracy)

c:\Users\ben81\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\ben81\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\ben81\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

Best parameters found:  {'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Test accuracy of best model:  0.7895


##### GradientBoosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

preds = model.predict(X_test)

accuracy = accuracy_score(y_test, preds)
print("Accuracy:", accuracy)

Accuracy: 0.785


##### SVM

In [ ]:
from sklearn.svm import SVC

svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
y_pred_svm
# y_pred_svm = svm_model.score(X_test,y_test)
# y_pred_svm

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

## Validation 1. Is Scaling meaningful?

### Experiment

In [8]:
X_raw = train.drop(columns=['Death'])
y_raw = train.loc[:,'Death']
X_train_not_scaled, X_test_not_scaled, y_train_not_scaled, y_test_not_scaled = train_test_split(X_raw, y_raw, test_size=0.2, random_state=42,stratify=y_raw)

#### Not scaled

##### XGBoost

In [9]:
dtrain_not_scaled = xgb.DMatrix(X_train_not_scaled, label=y_train_not_scaled)
dtest_not_scaled = xgb.DMatrix(X_test_not_scaled, label=y_test_not_scaled)

# params = {
#     'max_depth': 6, 
#     'eta': 0.3,  
#     'objective': 'binary:logistic',
#     'eval_metric': 'error' 
# }
params = {
           'learning_rate': 0.1, 
           'max_depth': 10, 
           'min_child_weight': 3, 
           'subsample': 0.9,
           'validate_parameters': 1,
           'scale_pos_weight': 1,
           'colsample_bytree': 0.7,
           }


num_rounds = 2000
model_not_scaled = xgb.train(params, dtrain_not_scaled, num_rounds)

preds = model_not_scaled.predict(dtest_not_scaled)
pred_labels_not_scaled = [round(pred) for pred in preds]

accuracy_not_scaled = accuracy_score(y_test_not_scaled, pred_labels_not_scaled)
print("Accuracy:", accuracy_not_scaled)
Counter(pred_labels_not_scaled)


Accuracy: 0.7615


Counter({0: 1932, 1: 68})

##### SVM

In [11]:
SVM_not_scaled = SVC()
SVM_not_scaled.fit(X_train_not_scaled,y_train_not_scaled)
print(SVM_not_scaled.score(X_test_not_scaled, y_test_not_scaled))
Counter(SVM_not_scaled.predict(X_test_not_scaled))

0.7895


Counter({0: 2000})

#### Scaled

In [12]:
bool_col_train = []
nume_col_train = []
bool_col_test = []
nume_col_test = []

for col in train.drop(columns=['Death']).columns:
    unique_values = train[col].unique()
    if len(unique_values) == 2 and all(value in [0, 1] for value in unique_values):
        bool_col_train.append(col)
    else:
        nume_col_train.append(col)

for col in test.columns:
    unique_values = test[col].unique()
    if len(unique_values) == 2 and all(value in [0, 1] for value in unique_values):
        bool_col_test.append(col)
    else:
        nume_col_test.append(col)


In [13]:
nume_col_train, bool_col_train

(['AGE', 'Height', 'Weight', 'FEV1_FVC_P', 'DLCO_VA_P', 'Survival period'],
 ['Adenocarcinoma',
  'Large cell carcinoma',
  'Squamous cell carcinoma',
  'TX',
  'T0',
  'T1',
  'T1a',
  'T1b',
  'T1c',
  'T2',
  'T2a',
  'T2b',
  'T3',
  'T4',
  'N1',
  'N2',
  'N3',
  'M1a',
  'M1b',
  'M1c',
  'Operation',
  'Chemotherapy',
  'Radiation Therapy',
  'Type of Drink_1',
  'Type of Drink_2',
  'Type of Drink_3',
  'Type of Drink_99',
  'Smoke_0',
  'Smoke_1',
  'Smoke_2',
  'EGFR mutation Detection_0',
  'EGFR mutation Detection_1',
  'EGFR mutation Detection_99'])

In [14]:
train_copy = train.copy()
test_copy = test.copy()

In [15]:
scaler = MinMaxScaler()
for col in nume_col_train:
    npy = train[col].to_numpy().reshape(-1,1)
    normalized_data = scaler.fit_transform(npy)
    train_copy[col] = normalized_data

for col in nume_col_test:
    npy = test[col].to_numpy().reshape(-1,1)
    normalized_data = scaler.fit_transform(npy)
    test_copy[col] = normalized_data


In [16]:
train_copy.columns

Index(['AGE', 'Adenocarcinoma', 'Large cell carcinoma',
       'Squamous cell carcinoma', 'TX', 'T0', 'T1', 'T1a', 'T1b', 'T1c', 'T2',
       'T2a', 'T2b', 'T3', 'T4', 'N1', 'N2', 'N3', 'M1a', 'M1b', 'M1c',
       'Height', 'Weight', 'FEV1_FVC_P', 'DLCO_VA_P', 'Operation',
       'Chemotherapy', 'Radiation Therapy', 'Survival period', 'Death',
       'Type of Drink_1', 'Type of Drink_2', 'Type of Drink_3',
       'Type of Drink_99', 'Smoke_0', 'Smoke_1', 'Smoke_2',
       'EGFR mutation Detection_0', 'EGFR mutation Detection_1',
       'EGFR mutation Detection_99'],
      dtype='object')

In [17]:
test_copy.columns

Index(['AGE', 'Adenocarcinoma', 'Large cell carcinoma',
       'Squamous cell carcinoma', 'TX', 'T0', 'T1', 'T1a', 'T1b', 'T1c', 'T2',
       'T2a', 'T2b', 'T3', 'T4', 'N1', 'N2', 'N3', 'M1a', 'M1b', 'M1c',
       'Height', 'Weight', 'FEV1_FVC_P', 'DLCO_VA_P', 'Operation',
       'Chemotherapy', 'Radiation Therapy', 'Survival period',
       'Type of Drink_1', 'Type of Drink_2', 'Type of Drink_3',
       'Type of Drink_99', 'Smoke_0', 'Smoke_1', 'Smoke_2',
       'EGFR mutation Detection_0', 'EGFR mutation Detection_1',
       'EGFR mutation Detection_99'],
      dtype='object')

In [18]:
X_preprocessed = train_copy.drop(columns=['Death'])
y_preprocessed = train_copy.loc[:,'Death']

X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(X_preprocessed, y_preprocessed, test_size=0.2, random_state=42,stratify=y_preprocessed)

##### XGBoost

In [19]:
dtrain_scaled = xgb.DMatrix(X_train_scaled, label=y_train_scaled)
dtest_scaled = xgb.DMatrix(X_test_scaled, label=y_test_scaled)

# params = {
#     'max_depth': 6, 
#     'eta': 0.3, 
#     'objective': 'binary:logistic',
#     'eval_metric': 'error'
# }

params = {
           'learning_rate': 0.1, 
           'max_depth': 10, 
           'min_child_weight': 3, 
           'subsample': 0.9,
           'validate_parameters': 1,
           'scale_pos_weight': 1,
           'colsample_bytree': 0.7,
           }

num_rounds = 100
model_scaled = xgb.train(params, dtrain_scaled, num_rounds)

preds = model_scaled.predict(dtest_scaled)
pred_labels_scaled = [round(pred) for pred in preds]

accuracy_scaled = accuracy_score(y_test_scaled, pred_labels_scaled)
print("Accuracy:", accuracy_scaled)
Counter(pred_labels_scaled)


Accuracy: 0.7655


Counter({0: 1928, 1: 72})

##### SVM

In [20]:
SVM_scaled = SVC()
SVM_scaled.fit(X_train_scaled,y_train_scaled)
print(SVM_scaled.score(X_test_scaled, y_test_scaled))
Counter(SVM_scaled.predict(X_test_scaled))

0.7895


Counter({0: 2000})

##### MLP

In [21]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu',solver='sgd',alpha=0.01, batch_size=64,
                   learning_rate_init=0.001, max_iter=1000)

In [22]:
mlp.fit(X_train_scaled,y_train_scaled)

MLPClassifier(alpha=0.01, batch_size=64, max_iter=1000, solver='sgd')

In [23]:
mlp.score(X_test_scaled, y_test_scaled)

0.7895

In [24]:
set(mlp.predict(X_test_scaled))

{0}

## Validation 2. Train with high-related columns

In [26]:
related_factors = [
    'Adenocarcinoma',
    'Large cell carcinoma',
    'Squamous cell carcinoma',
    'TX', 'T0', 'T1', 'T1a', 'T1b', 'T1c', 'T2', 'T2a', 'T2b', 'T3', 'T4',
    'N1', 'N2', 'N3',
    'M1a', 'M1b', 'M1c',
    'Smoke',
    'FEV1_FVC_P',
    'DLCO_VA_P',
    'EGFR mutation Detection',
    'Operation',
    'Chemotherapy',
    'Radiation Therapy',
    'Survival period',
    'Death'
]
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train = train.iloc[:,1:]
test = test.iloc[:,1:]

train_related = train[related_factors]
test_related = test[related_factors[:-1]]

In [27]:
train_related = pd.get_dummies(train_related, columns = [ 'Smoke', 'EGFR mutation Detection'])
test_related = pd.get_dummies(test_related, columns = [ 'Smoke', 'EGFR mutation Detection'])

In [28]:
train_related.columns

Index(['Adenocarcinoma', 'Large cell carcinoma', 'Squamous cell carcinoma',
       'TX', 'T0', 'T1', 'T1a', 'T1b', 'T1c', 'T2', 'T2a', 'T2b', 'T3', 'T4',
       'N1', 'N2', 'N3', 'M1a', 'M1b', 'M1c', 'FEV1_FVC_P', 'DLCO_VA_P',
       'Operation', 'Chemotherapy', 'Radiation Therapy', 'Survival period',
       'Death', 'Smoke_0', 'Smoke_1', 'Smoke_2', 'EGFR mutation Detection_0',
       'EGFR mutation Detection_1', 'EGFR mutation Detection_99'],
      dtype='object')

In [29]:
test_related.columns

Index(['Adenocarcinoma', 'Large cell carcinoma', 'Squamous cell carcinoma',
       'TX', 'T0', 'T1', 'T1a', 'T1b', 'T1c', 'T2', 'T2a', 'T2b', 'T3', 'T4',
       'N1', 'N2', 'N3', 'M1a', 'M1b', 'M1c', 'FEV1_FVC_P', 'DLCO_VA_P',
       'Operation', 'Chemotherapy', 'Radiation Therapy', 'Survival period',
       'Smoke_0', 'Smoke_1', 'Smoke_2', 'EGFR mutation Detection_0',
       'EGFR mutation Detection_1', 'EGFR mutation Detection_99'],
      dtype='object')

In [30]:
X_raw = train_related.drop(columns=['Death'])
y_raw = train_related.loc[:,'Death']
X_train_related, X_test_related, y_train_related, y_test_related = train_test_split(X_raw, y_raw, test_size=0.2, random_state=42,stratify=y_raw)

In [31]:
dtrain_related = xgb.DMatrix(X_train_related, label=y_train_related)
dtest_related = xgb.DMatrix(X_test_related, label=y_test_related)

# params = {
#     'max_depth': 6, 
#     'eta': 0.3,  
#     'objective': 'binary:logistic',
#     'eval_metric': 'error' 
# }
params = {
           'learning_rate': 0.1, 
           'max_depth': 10, 
           'min_child_weight': 3, 
           'subsample': 0.9,
           'validate_parameters': 1,
           'scale_pos_weight': 1,
           'colsample_bytree': 0.7,
           }


num_rounds = 2000
model_related = xgb.train(params, dtrain_related, num_rounds)

preds = model_related.predict(dtest_related)
pred_labels_related = [round(pred) for pred in preds]

accuracy_related = accuracy_score(y_test_related, pred_labels_related)
print("Accuracy:", accuracy_related)
Counter(pred_labels_related)


Accuracy: 0.7545


Counter({0: 1862, 1: 138})

## Submission

In [32]:
test

,AGE,Adenocarcinoma,Large cell carcinoma,Squamous cell carcinoma,TX,T0,T1,T1a,T1b,T1c,...,Smoke,Height,Weight,FEV1_FVC_P,DLCO_VA_P,EGFR mutation Detection,Operation,Chemotherapy,Radiation Therapy,Survival period
0,35,1,0,0,0,0,0,0,0,0,...,1,174.2,69.1,82,43,0,0,1,0,355
1,60,1,0,0,0,0,0,0,1,0,...,0,158.8,67.4,52,86,0,0,0,1,166
2,45,1,1,1,1,0,0,0,0,0,...,0,159.7,71.2,49,68,99,1,1,1,623
3,31,0,0,0,1,0,0,0,0,0,...,1,153.6,71.0,82,87,1,1,0,0,445
4,55,0,0,1,0,0,0,1,0,0,...,1,163.6,72.0,68,127,99,0,1,0,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,47,1,1,0,0,0,0,0,0,1,...,0,147.6,79.0,40,79,99,1,0,0,104
4996,44,1,0,1,0,0,1,0,0,0,...,2,151.0,41.6,69,67,99,0,0,0,42
4997,80,0,0,1,0,0,0,1,0,0,...,0,157.8,58.2,40,137,0,1,1,0,190
4998,69,1,0,0,0,0,0,0,0,0,...,2,167.5,56.7,89,77,0,1,1,1,635


In [33]:
# submission = pd.read_csv('submission.csv')
# submission['Death'] = ...
# submission['Death'].unique()

### XGBoost_scaled

In [34]:
real_test_not_scaled = xgb.DMatrix(test_copy)
preds =  model_not_scaled.predict(real_test_not_scaled)
pred_labels = [round(pred) for pred in preds]
Counter(pred_labels)

Counter({0: 3269, 1: 1731})

In [35]:
submission = pd.read_csv('submission.csv')
submission['Death'] = pred_labels
Counter(submission['Death'])

Counter({0: 3269, 1: 1731})

In [36]:
submission.to_csv('submission.csv',index=False)

### XGBoost_related

In [37]:
real_test_related = xgb.DMatrix(test_related)
preds =  model_related.predict(real_test_related)
pred_labels = [round(pred) for pred in preds]
Counter(pred_labels)

Counter({0: 4549, 1: 451})

In [38]:
submission = pd.read_csv('submission.csv')
submission['Death'] = pred_labels
Counter(submission['Death'])

Counter({0: 4549, 1: 451})

In [39]:
submission.to_csv('submission.csv',index=False)